This example shows how to construct a CMD model from SFRs.
We will use PARESC stellar models and YBC bolometric corrections to sample the isochrones and assume a Kroupa (2001) IMF.

In [ ]:
import StarFormationHistories as SFH
using StellarTracks: PARSECLibrary, isochrone
using BolometricCorrections: YBCGrid, filternames
using InitialMassFunctions: Kroupa2001
using TypedTables: Table, getproperties
using StaticArrays: SVector

The functions for generating mock CMDs for complex stellar populations take the mass in each SSP, not the SFR. To use SFRs, they need to be integrated over time bins to get the total mass per SSP.

In [70]:
ages = range(13.7e9, 0.1e9; step = -0.1e9) # Ages of SSPs in yr from 13.7 Gyr to 0.1 Gyr
sfrs = rand(length(ages)) .* 1e-3  # Random SFRs in solar masses / yr
# Need to define left bin edge for integrating SFRs;
# i.e., the first SFR bin goes from 13.76 -- 13.7 Gyr, next goes from 13.7 -- 13.6, and so on
T_max = 13.76e9
# Other properties for the stellar population
dmod = 25.0 # Distance modulus to add to sampled CMD
Av = 0.0 # V-band extinction to add to CMD; we will not add any

0.0

In [71]:
# Define the bins over which the SFR will be integrated, now in units of years
age_l = ages
age_h = vcat(T_max, age_l[begin:end-1])
[age_l age_h] # show bins

137×2 Matrix{Float64}:
 1.37e10  1.376e10
 1.36e10  1.37e10
 1.35e10  1.36e10
 1.34e10  1.35e10
 1.33e10  1.34e10
 1.32e10  1.33e10
 1.31e10  1.32e10
 1.3e10   1.31e10
 1.29e10  1.3e10
 1.28e10  1.29e10
 1.27e10  1.28e10
 1.26e10  1.27e10
 1.25e10  1.26e10
 ⋮        
 1.2e9    1.3e9
 1.1e9    1.2e9
 1.0e9    1.1e9
 9.0e8    1.0e9
 8.0e8    9.0e8
 7.0e8    8.0e8
 6.0e8    7.0e8
 5.0e8    6.0e8
 4.0e8    5.0e8
 3.0e8    4.0e8
 2.0e8    3.0e8
 1.0e8    2.0e8

With the time bins set up, we can now calculate the total stellar mass formed in each time bin by integrating, assuming fixed SFR within each bin.

In [72]:
ssp_masses = [sfrs[i] * (age_h[i] - age_l[i]) for i in eachindex(sfrs)]

137-element Vector{Float64}:
 10018.116480197976
  5541.288411233081
 85689.1891594672
 40408.24170991206
 43420.96297598789
  3269.6308354220773
 51868.57506747481
 59782.632268777175
 29885.39210282616
   879.1405174133461
 57691.55614294534
 81361.27185785041
  6388.775014704706
     ⋮
 87938.41149198785
 51379.271856562096
 88314.96699366538
 28241.201445019593
 70432.4913196472
 79481.03362213884
 59679.90019316475
 18224.539164488742
 65198.918884571096
 84318.89561584096
  8253.701966132488
 30107.293221860844

Now, assuming you have a vector for the metallicity at each SSP age, you can interpolate isochrones on-the-fly with StellarTracks.jl and BolometricCorrections.jl.

In [73]:
# Simple linear AMR model for simplicity; replace with whatever you want, same length as ages
MH = @. 0.1 * (13.76 .- (ages/1e9)) + -2.0

-1.994:0.01:-0.6339999999999999

In [74]:
# Load data for isochrone interpolation
tracklib = PARSECLibrary()
bcg = YBCGrid("acs_wfc"); # We'll use HST/ACS filters here

In [ ]:
# Now interpolate appropriate isochrones. SSP ages have to be converted to logarithmic ages log10(age [yr]).
isos = [isochrone(tracklib, bcg, log10(ages[i]), MH[i], Av) for i in eachindex(ages)]

137-element Vector{Table{@NamedTuple{eep::Int64, m_ini::Float64, logTe::Float64, Mbol::Float64, logg::Float64, C_O::Float64, F435W::Float64, F475W::Float64, F502N::Float64, F550M::Float64, F555W::Float64, F606W::Float64, F625W::Float64, F658N::Float64, F660N::Float64, F775W::Float64, F814W::Float64, F850LP::Float64}, 1, @NamedTuple{eep::Vector{Int64}, m_ini::Vector{Float64}, logTe::Vector{Float64}, Mbol::Vector{Float64}, logg::Vector{Float64}, C_O::Vector{Float64}, F435W::SubArray{Float64, 1, Matrix{Float64}, Tuple{Base.Slice{Base.OneTo{Int64}}, Int64}, true}, F475W::SubArray{Float64, 1, Matrix{Float64}, Tuple{Base.Slice{Base.OneTo{Int64}}, Int64}, true}, F502N::SubArray{Float64, 1, Matrix{Float64}, Tuple{Base.Slice{Base.OneTo{Int64}}, Int64}, true}, F550M::SubArray{Float64, 1, Matrix{Float64}, Tuple{Base.Slice{Base.OneTo{Int64}}, Int64}, true}, F555W::SubArray{Float64, 1, Matrix{Float64}, Tuple{Base.Slice{Base.OneTo{Int64}}, Int64}, true}, F606W::SubArray{Float64, 1, Matrix{Float64}, 

With the isochrones interpolated, we now have everything we need to construct the CMD. We will prepare inputs for `generate_stars_mass_composite`, then call it.

In [96]:
# Selects photometry columns from interpolated isochrone
function select_mags(iso, bcg)
    return select_mags(getproperties(iso, filternames(bcg)))
end
# Converts output from above into more efficient representation
function select_mags(subtable::Table{NamedTuple{B,NTuple{C,D}}}) where {B, C, D}
    return [SVector{C}(values(subtable[i])) for i in eachindex(subtable)]
end
# select_mags(isos[1], bcg)
mini_vec = [i.m_ini for i in isos] # Initial mass vectors from isochrones
mags = [select_mags(i, bcg) for i in isos] # Photometric magnitudes from isochrones
filters = collect(string.(filternames(bcg))) # Filter names for each photometric magnitude
mstar = sum(ssp_masses) # Total stellar mass
ssp_massfracs = ssp_masses ./ mstar # generate_stars_mass_composite requires fraction of mass in each SSP
imf = Kroupa2001(0.08, 100.0) # Kroupa 2001 IMF with minimum stellar mass 0.08 M⊙ and maximum stellar mass 100 M⊙
binary_model = SFH.RandomBinaryPairs(0.4) # Model for including unresolved photometric binaries
mag_lim, mag_lim_name = 32.0, "F606W" # Sampled stars will only be returned if they are brighter than 32 mag in F606W (more efficient)
stars = SFH.generate_stars_mass_composite(mini_vec, mags, filters, mstar, ssp_massfracs, imf; 
            binary_model=binary_model, dist_mod=dmod, mag_lim=mag_lim, mag_lim_name=mag_lim_name)

(Vector{SVector{2, Float64}}[[[0.5496767951098352, 0.28103505781909804], [0.5363960952218526, 0.0], [0.685147471246987, 0.08611583588414196], [0.5297909468367197, 0.0], [1.226039049908765, 0.3493371410419584], [0.8771328450955185, 0.18004982397953642], [1.4379502818350902, 0.46135640993434246], [1.072345100033884, 0.3318708026378101], [1.4843228385760479, 0.22843004711882214], [0.7482566588949615, 0.0]  …  [0.6520538921102046, 0.609452738130978], [0.6551386004277142, 0.0], [0.8793598291978821, 0.4527904247329999], [0.6574953722197125, 0.0], [0.5123168401035736, 0.14704981555077273], [0.5627493965383082, 0.0], [1.6921827996773737, 0.20802855025712233], [8.316661110045153, 0.6038907983890038], [0.5404003821293645, 0.0], [62.638900953809326, 0.5369072981590597]], [[0.6307033108023503, 0.08795063568387236], [0.5880703483881695, 0.24270279469993986], [0.5529736152294642, 0.0], [0.7287889313331204, 0.08339653107073135], [1.2769205491078774, 0.33299273359415255], [0.7602171542355666, 0.095563

Now stars[1] will contain initial mass information of the samples stars and stars[2] will contain photometry.

In [97]:
# Concatenate along first axis (combines results from all SSPs into one vector.
# The first entry in each element is primary star mass, second entry is secondary star mass (if binary)
stars_mini = reduce(vcat, stars[1])

1506076-element Vector{SVector{2, Float64}}:
 [0.5496767951098352, 0.28103505781909804]
 [0.5363960952218526, 0.0]
 [0.685147471246987, 0.08611583588414196]
 [0.5297909468367197, 0.0]
 [1.226039049908765, 0.3493371410419584]
 [0.8771328450955185, 0.18004982397953642]
 [1.4379502818350902, 0.46135640993434246]
 [1.072345100033884, 0.3318708026378101]
 [1.4843228385760479, 0.22843004711882214]
 [0.7482566588949615, 0.0]
 [0.5185931564922884, 0.14477761593588018]
 [0.7222930421032567, 0.4758268024658731]
 [1.3104651981655697, 0.5001679849924234]
 ⋮
 [0.6682960316303579, 0.46126170315752923]
 [0.6473870741327187, 0.6087523492944038]
 [0.9383630665852071, 0.0]
 [0.7268277474071551, 0.5065093382955063]
 [0.9047777969157033, 0.11563354337328152]
 [1.3092589904533256, 0.0]
 [1.2888418426168902, 0.250418331363466]
 [3.279074856979736, 0.0]
 [2.858555063058428, 0.250094993007444]
 [0.8452184230665828, 0.2149066064659971]
 [1.828667511192542, 0.0]
 [1.4767260302300325, 0.0]

In [98]:
stars_mags = reduce(vcat, stars[2])

1506076-element Vector{SVector{12, Float64}}:
 [32.411704848718195, 32.11496520558554, 31.939100658997724, 31.591134772651543, 31.717381481534638, 31.457383176309627, 31.258250947433748, 31.016989362892012, 31.080279824975715, 30.821089962989184, 30.761450174080338, 30.606911189983506]
 [32.617747322799225, 32.31929940577038, 32.13983513015743, 31.8005425827154, 31.924155033973783, 31.66964812962599, 31.47380008415217, 31.235244550167444, 31.29820035626028, 31.04519834297125, 30.98739214310588, 30.83745872855919]
 [30.835510059536297, 30.653709639088788, 30.530094510120467, 30.29149813102105, 30.378418858368107, 30.19873892309784, 30.051950555335324, 29.860424946365086, 29.912508715395816, 29.727650270440527, 29.688276965462038, 29.585599813497637]
 [32.69409616281741, 32.39069999662431, 32.20931109041481, 31.86515857340786, 31.9905530497562, 31.732627177783776, 31.534661799407083, 31.2942637399281, 31.35765047999712, 31.101613344967866, 31.042977080038895, 30.89092916004981]
 [34.8957

In [ ]:
# We can combine these into a TypedTables.Table,
mass_table = Table(NamedTuple{(:primary_mass, :secondary_mass)}(Tuple(i)) for i in stars_mini)

Table with 2 columns and 1506076 rows:
      primary_mass  secondary_mass
    ┌─────────────────────────────
 1  │ 0.549677      0.281035
 2  │ 0.536396      0.0
 3  │ 0.685147      0.0861158
 4  │ 0.529791      0.0
 5  │ 1.22604       0.349337
 6  │ 0.877133      0.18005
 7  │ 1.43795       0.461356
 8  │ 1.07235       0.331871
 9  │ 1.48432       0.22843
 10 │ 0.748257      0.0
 11 │ 0.518593      0.144778
 12 │ 0.722293      0.475827
 13 │ 1.31047       0.500168
 14 │ 0.698258      0.195926
 15 │ 21.6496       0.294597
 16 │ 0.672532      0.473318
 17 │ 2.6738        0.256639
 18 │ 0.685169      0.0
 19 │ 4.52935       0.283865
 20 │ 0.761817      0.0
 21 │ 1.12855       0.291025
 22 │ 2.10228       0.285659
 23 │ 0.637178      0.274167
 ⋮  │      ⋮              ⋮

In [101]:
mag_table = Table(NamedTuple{filternames(bcg)}(Tuple(i)) for i in stars_mags)

Table with 12 columns and 1506076 rows:
      F435W    F475W    F502N    F550M    F555W    F606W    F625W    F658N    ⋯
    ┌──────────────────────────────────────────────────────────────────────────
 1  │ 32.4117  32.115   31.9391  31.5911  31.7174  31.4574  31.2583  31.017   ⋯
 2  │ 32.6177  32.3193  32.1398  31.8005  31.9242  31.6696  31.4738  31.2352  ⋯
 3  │ 30.8355  30.6537  30.5301  30.2915  30.3784  30.1987  30.052   29.8604  ⋯
 4  │ 32.6941  32.3907  32.2093  31.8652  31.9906  31.7326  31.5347  31.2943  ⋯
 5  │ 34.8958  34.484   34.2977  33.7584  33.9442  33.5695  33.3078  33.0165  ⋯
 6  │ 37.0555  36.5586  36.4041  35.6518  35.8908  35.4054  35.09    34.7535  ⋯
 7  │ 33.5733  33.2223  33.0328  32.6173  32.766   32.4637  32.2404  31.9794  ⋯
 8  │ 35.102   34.6807  34.4949  33.9364  34.1279  33.7419  33.4742  33.1782  ⋯
 9  │ 36.3988  35.9246  35.7543  35.0694  35.2938  34.8404  34.5392  34.2155  ⋯
 10 │ 29.3574  29.2148  29.1134  28.9156  28.9881  28.8389  28.7138  28.5473  ⋯


In [102]:
# Concatenate the tables
stars_table = Table(mass_table, mag_table)

Table with 14 columns and 1506076 rows:
      primary_mass  secondary_mass  F435W    F475W    F502N    F550M    ⋯
    ┌────────────────────────────────────────────────────────────────────
 1  │ 0.549677      0.281035        32.4117  32.115   31.9391  31.5911  ⋯
 2  │ 0.536396      0.0             32.6177  32.3193  32.1398  31.8005  ⋯
 3  │ 0.685147      0.0861158       30.8355  30.6537  30.5301  30.2915  ⋯
 4  │ 0.529791      0.0             32.6941  32.3907  32.2093  31.8652  ⋯
 5  │ 1.22604       0.349337        34.8958  34.484   34.2977  33.7584  ⋯
 6  │ 0.877133      0.18005         37.0555  36.5586  36.4041  35.6518  ⋯
 7  │ 1.43795       0.461356        33.5733  33.2223  33.0328  32.6173  ⋯
 8  │ 1.07235       0.331871        35.102   34.6807  34.4949  33.9364  ⋯
 9  │ 1.48432       0.22843         36.3988  35.9246  35.7543  35.0694  ⋯
 10 │ 0.748257      0.0             29.3574  29.2148  29.1134  28.9156  ⋯
 11 │ 0.518593      0.144778        32.8241  32.5126  32.3282  31.9756  

From here you can apply completeness and photometric error models with `StarFormationHistories.model_cmd` if you wish.

In [105]:
# This can now be written to a file, for example with CSV.jl
# formatting options exist (e.g., to reduce precision of numbers in the output)
# import CSV
# CSV.write("output.txt", stars_table; delim=' ')